In [1]:
# 
# import json
# from datetime import datetime
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Download the webpage

In [2]:
url = "https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/Monkeypox-Data.aspx"

In [3]:
page = requests.get(url)

### Parse the webpage

In [4]:
soup = BeautifulSoup( page.content, 'html.parser' )

In [5]:
content = soup.find("div", id="DeltaPlaceHolderMain")

In [6]:
tables = content.find_all("table")

In [7]:
len(tables)

8

In [8]:
tables[2].find( "th" ).find("strong").text

'By Local\r\n  Health Jurisdiction'

In [37]:
for tag in content.find_all(text=re.compile('Jurisdiction')):
    table = tag.findParent("table")

In [38]:
def safetxt(element):
    v = element.text.strip()
    v = v.replace("\u200b", "")
    return v

In [39]:
def safenumber(element):
    v = safetxt(element)
    v = v.replace(",", "")
    v = v.replace(" ", "")
    return v

In [52]:
dict_list = []

for row in table.find_all("tr"):
    cell_list = row.find_all("td")
    #print(cell_list)
    try:
        d = dict(
            lhj_name = safetxt( cell_list[0] ),
            cases = safenumber( cell_list[1] )        
        )
        dict_list.append(d)
    except Exception as e: 
        print(e)

list index out of range


### Parsing date

In [79]:
soup = BeautifulSoup( page.content, 'html.parser' )

In [125]:
date = soup.find_all("p")

In [126]:
print(date)

[<p></p>, <p style="line-height: 1.6;"><span style="color: #0072c6; font-size: 36px; font-weight: 700; background-color: transparent;">Monkeypox Data in California </span><br/></p>, <p style="line-height: 1.6;"><span class="ms-rteStyle-Emphasis">Data are updated on Tuesdays and Thursdays. Last updated August 4, 2022.</span></p>, <p></p>, <p style="line-height: 1.6;"><br/></p>, <p></p>, <p><br/></p>, <p><br/></p>, <p><br/></p>, <p></p>, <p>
                                        PO Box 997377<br/>
                                        MS 0500<br/>
                                        Sacramento, CA 95899-7377<br/>
</p>, <p>
<strong>For General Public Information:</strong><br/>
<span>(916) 558-1784</span>
</p>, <p>
<strong>COVID 19 Information Line:</strong><br/>
<!-- <span>1-833-544-2374 </span> -->
<span>1-833-4CA4ALL</span><br/>
<span>(1-833-422-4255)<span>
</span></span></p>]


In [ ]:
spans = soup.find_all('span', attrs={'class':'ms-rteStyle-Emphasis'})
for span in spans:
    spans.append(str)
    print(spans)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
type(spans)

In [187]:
print(spans)

[<span class="ms-rteStyle-Emphasis">Data are updated on Tuesdays and Thursdays. Last updated August 4, 2022.</span>]


In [198]:
date_final = soup.find_all(spans.strip("Data are updated on Tuesdays and Thursdays. Last updated"))

AttributeError: ResultSet object has no attribute 'strip'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

### Isolate Date

In [183]:
"""years = r'((?:22|23)\d\d)'
pattern = r'(%%s) +(%%s), *%s' % years

thirties = pattern % (
     "September|April|June|November",
     r'0?[1-9]|[12]\d|30')

thirtyones = pattern % (
     "January|March|May|July|August|October|December",
     r'0?[1-9]|[12]\d|3[01]')

fours = '(?:%s)' % '|'.join('%02d' % x for x in range(4, 100, 4))

feb = r'(February) +(?:%s|%s)' % (
     r'(?:(0?[1-9]|1\d|2[0-8])), *%s' % years, # 1-28 any year
     r'(?:(29), *((?:(?:22|23)%s)|2000))' % fours)  # 29 leap years only

result = '|'.join('(?:%s)' % x for x in (thirties, thirtyones, feb))
r = re.compile(result)
print(result)"""

'years = r\'((?:22|23)\\d\\d)\'\npattern = r\'(%%s) +(%%s), *%s\' % years\n\nthirties = pattern % (\n     "September|April|June|November",\n     r\'0?[1-9]|[12]\\d|30\')\n\nthirtyones = pattern % (\n     "January|March|May|July|August|October|December",\n     r\'0?[1-9]|[12]\\d|3[01]\')\n\nfours = \'(?:%s)\' % \'|\'.join(\'%02d\' % x for x in range(4, 100, 4))\n\nfeb = r\'(February) +(?:%s|%s)\' % (\n     r\'(?:(0?[1-9]|1\\d|2[0-8])), *%s\' % years, # 1-28 any year\n     r\'(?:(29), *((?:(?:22|23)%s)|2000))\' % fours)  # 29 leap years only\n\nresult = \'|\'.join(\'(?:%s)\' % x for x in (thirties, thirtyones, feb))\nr = re.compile(result)\nprint(result)'

In [181]:
#date = span.find_all(text=r)

In [182]:
#

[]


### Convert to dataframe

In [46]:
df = pd.DataFrame(dict_list)

### Export

In [49]:
df.to_csv("raw/cdph/latest.csv", index=False)